# 6. 데이터 정제, 탐색 및 시리즈 연산
## (1) 판다스 시리즈에서 값을 얻기
- 데이터 : 미국 종단 조사(NLS)의 고등학교 내신성적(GPA) 사용

In [2]:
import pandas as pd
nls97 = pd.read_csv('data/nls97b.csv')
nls97.set_index('personid', inplace=True)

In [6]:
gpaoverall = nls97.gpaoverall
type(gpaoverall)

pandas.core.series.Series

In [7]:
gpaoverall.head()

personid
100061    3.06
100139     NaN
100284     NaN
100292    3.45
100583    2.91
Name: gpaoverall, dtype: float64

In [8]:
gpaoverall.index

Int64Index([100061, 100139, 100284, 100292, 100583, 100833, 100931, 101089,
            101122, 101132,
            ...
            998997, 999031, 999053, 999087, 999103, 999291, 999406, 999543,
            999698, 999963],
           dtype='int64', name='personid', length=8984)

In [10]:
# 시리즈 슬라이싱 : 5개 값
gpaoverall[:5]

personid
100061    3.06
100139     NaN
100284     NaN
100292    3.45
100583    2.91
Name: gpaoverall, dtype: float64

In [11]:
gpaoverall.tail()

personid
999291    3.11
999406    2.17
999543     NaN
999698     NaN
999963    3.78
Name: gpaoverall, dtype: float64

In [12]:
gpaoverall[-5:]

personid
999291    3.11
999406    2.17
999543     NaN
999698     NaN
999963    3.78
Name: gpaoverall, dtype: float64

### loc 접근자
- loc 접근자에 인덱스 레이블을 전달하면 '스칼라 값'이 반환됨
- 인덱스 레이블의 리스트를 전달하면, 값이 하나이든 여러개이든 시리즈를 얻음

In [14]:
# loc 접근자 사용
gpaoverall.loc[100061]

3.06

In [15]:
gpaoverall.loc[[100061]]

personid
100061    3.06
Name: gpaoverall, dtype: float64

In [16]:
gpaoverall.loc[100061:100833]

personid
100061    3.06
100139     NaN
100284     NaN
100292    3.45
100583    2.91
100833    2.46
Name: gpaoverall, dtype: float64

### iloc 접근자
- loc와 달리 레이블이 아닌 행번호의 리스트를 취함

In [17]:
gpaoverall.iloc[[0]]

personid
100061    3.06
Name: gpaoverall, dtype: float64

In [18]:
gpaoverall.iloc[[0, 1, 2, 3, 4]]

personid
100061    3.06
100139     NaN
100284     NaN
100292    3.45
100583    2.91
Name: gpaoverall, dtype: float64

In [19]:
gpaoverall.iloc[:5]

personid
100061    3.06
100139     NaN
100284     NaN
100292    3.45
100583    2.91
Name: gpaoverall, dtype: float64

## (2) 판다스 시리즈에 대한 요약통계 표시

In [20]:
import numpy as np

In [21]:
gpaoverall.mean()

2.8184077281812145

In [22]:
gpaoverall.describe()

count    6004.000000
mean        2.818408
std         0.616357
min         0.100000
25%         2.430000
50%         2.860000
75%         3.260000
max         4.170000
Name: gpaoverall, dtype: float64

In [23]:
gpaoverall.quantile(np.arange(0.1, 1.1, 0.1))

0.1    2.02
0.2    2.31
0.3    2.52
0.4    2.70
0.5    2.86
0.6    3.01
0.7    3.17
0.8    3.36
0.9    3.60
1.0    4.17
Name: gpaoverall, dtype: float64

### 시리즈의 부분집합
- between()사용

In [24]:
gpaoverall.loc[gpaoverall.between(3, 3.5)].head()

personid
100061    3.06
100292    3.45
101526    3.37
101527    3.26
102125    3.14
Name: gpaoverall, dtype: float64

In [25]:
gpaoverall.loc[(gpaoverall<2) | (gpaoverall>4)].sample(5, random_state=2)

personid
932782    1.90
561335    1.82
850001    4.10
292455    1.97
644271    1.97
Name: gpaoverall, dtype: float64

In [27]:
gpaoverall.loc[gpaoverall > gpaoverall.quantile(0.99)].agg(['count', 'min', 'max'])

count    60.00
min       3.98
max       4.17
Name: gpaoverall, dtype: float64

### 전체 값에 걸쳐 조건을 테스트(**any, all, sum)
- any : 시리즈에 속한 값이 하나라도 조건을 만족할 때 true
- all : 시리즈의 모든 값이 조건을 만족할 때 true
- sum : true값을 1로 해석해 true값의 개수를 얻음

In [29]:
# gpa가 4를 넘는 사람이 있는가?
(gpaoverall > 4).any()

True

In [30]:
# 모든 사람의 gpa가 0이상인가?
(gpaoverall >= 0).all()

False

In [31]:
# gpa가 0 이상인 사람 수
(gpaoverall >= 0).sum()

6004

In [32]:
# gpa가 0인 사람 수
(gpaoverall == 0).sum()

0

In [33]:
# gpa값이 없는 사람 수
gpaoverall.isnull().sum()

2980

### 시리즈에서 서로 다른 열의 값을 기준으로 한 부분집합들의 기술 통계
- 2016년 임금소득이 75백분위수를 넘는 개인과 25백분위수 아래인 개인의 고등학교 GPA평균을 각각 표시

In [36]:
nls97.loc[nls97.wageincome > nls97.wageincome.quantile(0.75), 'gpaoverall'].mean()

3.0804171011470256

In [37]:
nls97.loc[nls97.wageincome < nls97.wageincome.quantile(0.25), 'gpaoverall'].mean()

2.720143415906124

### 범주형 데이터를 포함하는 시리즈에 대한 기술과 빈도

In [38]:
nls97.maritalstatus.describe()

count        6672
unique          5
top       Married
freq         3066
Name: maritalstatus, dtype: object

In [39]:
nls97.maritalstatus.value_counts()

Married          3066
Never-married    2766
Divorced          663
Separated         154
Widowed            23
Name: maritalstatus, dtype: int64